# 3. プロンプトエンジニアリング


In [ ]:
import os

#### 【注意】既知のエラーについて

openai パッケージが依存する httpx のアップデートにより、`openai==1.40.6` を使用する箇所で `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'` というエラーが発生するようになりました。

このエラーは、`!pip install httpx==0.27.2` のように、httpx の特定バージョンをインストールすることで回避することができます。

なお、Google Colab で一度上記のエラーに遭遇したあとで `!pip install httpx==0.27.2` のようにパッケージをインストールし直した場合、以下のどちらかの操作を実施する必要があります。

- Google Colab の「ランタイム」から「セッションを再起動する」を実行する
- 「ランタイムを接続解除して削除」を実行してパッケージのインストールからやり直す


In [ ]:
%pip install httpx==0.27.2

## 3.2. プロンプトエンジニアリングとは


In [ ]:
import boto3

client = boto3.client("bedrock-runtime")

response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    messages=[
        {
            "role": "user",
            "content": [{"text": "プロンプトエンジニアリングとは"}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

In [ ]:
import boto3

client = boto3.client("bedrock-runtime")

response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "質問に100文字程度で答えてください。"}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "プロンプトエンジニアリングとは"}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

## 3.3. プロンプトの構成要素の基本


### プロンプトのテンプレート化


In [ ]:
prompt = '''\
以下の料理のレシピを考えてください。

料理名: """
{dish}
"""
'''


def generate_recipe(dish: str) -> str:
    response = client.converse(
        modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
        messages=[
            {
                "role": "user",
                "content": [{"text": prompt.format(dish=dish)}],
            }
        ]
    )

    response_text = response["output"]["message"]["content"][0]["text"]
    return response_text

recipe = generate_recipe("カレー")
print(recipe)

In [ ]:
prompt = """\
ユーザーが入力した料理のレシピを考えてください。

料理名: '''
{dish}
'''
"""


def generate_recipe(dish: str) -> str:
    response = client.converse(
        modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
        system=[{"text": "ユーザーが入力した料理のレシピを考えてください。"}],
        messages=[
            {
                "role": "user",
                "content": [{"text": f"{dish}"}],
            }
        ]
    )

    response_text = response["output"]["message"]["content"][0]["text"]
    return response_text

recipe = generate_recipe("カレー")
print(recipe)

### 出力形式を指定する


In [ ]:
system_prompt = """\
ユーザーが入力した料理のレシピを考えてください。

出力は以下のJSON形式にしてください。

```
{
  "材料": ["材料1", "材料2"],
  "手順": ["手順1", "手順2"]
}
```
"""

response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": system_prompt}],
    messages=[
        {
          "role": "user",
          "content": [{"text": "カレー"}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

## 3.4. プロンプトエンジニアリングの定番の手法


### Zero-shot プロンプティング


In [ ]:
response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "入力をポジティブ・ネガティブ・中立のどれかに分類してください。"}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "ChatGPTはとても便利だ"}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

### Few-shot プロンプティング


In [ ]:
response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "入力がAIに関係するか回答してください。"}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "ChatGPTはとても便利だ"}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

In [ ]:
response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "入力がAIに関係するか回答してください。"}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "AIの進化はすごい"}],
        },
        {
            "role": "assistant",
            "content": [{"text": "true"}],
        },
        {
            "role": "user",
            "content": [{"text": "今日は良い天気だ"}],
        },
        {
            "role": "assistant",
            "content": [{"text": "false"}],
        },
        {
            "role": "user",
            "content": [{"text": "ChatGPTはとても便利だ"}],
        },
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

### （コラム）Few-shot プロンプティングのその他の形式


In [ ]:
prompt = """\
入力がAIに関係するか回答してください。

Q: AIの進化はすごい
A: true
Q: 今日は良い天気だ
A: false
Q: ChatGPTはとても便利だ
A: 
"""

response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    messages=[
        {
            "role": "user",
            "content": [{"text": prompt}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

In [ ]:
response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "入力がAIに関係するか回答してください。"}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "AIの進化はすごい"}],
        },
        {
            "role": "assistant",
            "content": [{"text": "true"}],
        },
        {
            "role": "user",
            "content": [{"text": "今日は良い天気だ"}],
        },
        {
            "role": "assistant",
            "content": [{"text": "false"}],
        },
        {
            "role": "user",
            "content": [{"text": "ChatGPTはとても便利だ"}],
        },
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

### Zero-shot Chain-of-Thought プロンプティング


In [ ]:
response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "回答だけ一言で出力してください。"}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "10 + 2 * 3 - 4 * 2"}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

In [ ]:
response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "ステップバイステップで考えてください。"}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "10 + 2 * 3 - 4 * 2"}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)